# Building the hotdog / no hotdog classifier

Started on 13th June 2018. Let's see how far I get.

## Project flow:

1. Get training data manually: imagenet + google images search for 'hotdog'

2. Crop "manually" the google images and imagenet to have the same input dimensions

3. Train a first example

4. Refine the example

    * Write a writeup for presentations, blog posts, etc.

5. Automate the training data collection

    * Write a writeup for presentations, blog posts, etc.

6. Implement data augmentation

7. Play around with advanced concepts: Transfer learning, resizing like in fastai, test-time augmentation, stochastic GD with restarts...

    * Write a writeup for presentations, blog posts, etc.

8. Port the thing to Android?